In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 使用内置方法进行训练和评估

指南涵盖使用内置 API 进行训练和验证时需要的训练、评估和预测（推断）模型（例如 model.fit()、model.evaluate()、model.predict()）。

In [4]:
inputs=keras.Input(shape=(784,),name='digits')
x=layers.Dense(64,activation='relu',name='dense_1')(inputs)
x=layers.Dense(64,activation='relu',name='dense_2')(x)
outputs=layers.Dense(10,activation='softmax',name='predictions')(x)
model=keras.Model(inputs,outputs)

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess the data (these are NumPy arrays)
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
print("Fit model on training data")
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=2,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_val, y_val),
)


11493376/11490434 [==============================] - 11s 1us/step
Fit model on training data
Epoch 1/2
782/782 [==============================] - 6s 7ms/step - loss: 0.5903 - sparse_categorical_accuracy: 0.8358 - val_loss: 0.2152 - val_sparse_categorical_accuracy: 0.9337
Epoch 2/2
782/782 [==============================] - 4s 5ms/step - loss: 0.1877 - sparse_categorical_accuracy: 0.9455 - val_loss: 0.1349 - val_sparse_categorical_accuracy: 0.9601


In [5]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(x_test[:3])
print("predictions shape:", predictions.shape)


Evaluate on test data
79/79 [==============================] - 0s 3ms/step - loss: 0.1410 - sparse_categorical_accuracy: 0.9594
test loss, test acc: [0.14103305339813232, 0.9593999981880188]
Generate predictions for 3 samples
predictions shape: (3, 10)


# compile() 方法：指定损失、指标和优化器
要使用 fit() 训练模型，您需要指定损失函数、优化器以及一些要监视的指标（可选）。

In [ ]:
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)


metrics 参数应当为列表 -- 您的模型可以具有任意数量的指标。

如果模型具有多个输出，则可以为每个输出指定不同的损失和指标，并且可以调节每个输出对模型总损失的贡献。您可以在“将数据传递到多输入、多输出模型”部分中找到有关此内容的更多详细信息。

# 自定义损失
有两种方法来使用 Keras 提供自定义损失。第一个示例创建一个接受输入 y_true 和 y_pred 的函数。下面的示例显示了一个计算实际数据与预测值之间的均方误差的损失函数：

In [ ]:
def custom_mean_squared_error(y_true, y_pred):
    return tf.math.reduce_mean(tf.square(y_true - y_pred))


model = get_uncompiled_model()
model.compile(optimizer=keras.optimizers.Adam(), loss=custom_mean_squared_error)

# We need to one-hot encode the labels to use MSE
y_train_one_hot = tf.one_hot(y_train, depth=10)
model.fit(x_train, y_train_one_hot, batch_size=64, epochs=1)
